In [ ]:
import json
def flatten_dict(d, parent_key='', sep='_', parent_keys=set()):
    items = []
    for k, v in d.items():
        # Check if key is unique in the context of parent keys
        new_key = k if (k not in parent_keys) else f'{parent_key}{sep}{k}' if parent_key else k
        if isinstance(v, dict):
            # Update the set of parent keys for the next level
            new_parent_keys = parent_keys.union(set(d.keys()))
            items.extend(flatten_dict(v, new_key, sep, new_parent_keys).items())
        else:
            items.append((new_key, v))
    return dict(items)

def read_jsonl_and_flatten(file_path):
    flattened_data = []
    with open(file_path, 'r') as file:
        for line in file:
            # Load the line as a JSON object
            data_dict = json.loads(line)
            # Flatten the dictionary
            flattened_dict = flatten_dict(data_dict)
            
            # Might be better to clean this in another function
            # Check if the dictionary has the right number of keys
            if len(flattened_dict.keys()) != 34:
                print("Skipping due to wrong number of keys")
                print(flattened_dict.keys())
                print(flattened_dict)
                continue 

            if flattened_dict["jobs_towardsai_url"] == 'nan':
                print("Skipping due to nan url")
                print(flattened_dict)
                continue
            
            # Add the flattened dictionary to the list
            flattened_data.append(flattened_dict)

    return flattened_data

# Example usage
file_path = '../data/formatted_jobs_feb5_24_results.jsonl'
flattened_data = read_jsonl_and_flatten(file_path)
print("end of file")
print(flattened_data[0])
print(len(flattened_data[0].keys()))
print(flattened_data[9].keys())



In [ ]:
import math

def optimize_keys(data_dict):
    # Function to check if a value is valid (not None, Nan, empty string, or 'Not Specified')

    def is_valid(value):
        # Check for None
        if value is None:
            return False

        # Check for NaN for float values
        if isinstance(value, float) and math.isnan(value):
            return False

        # Check for empty strings and lists
        if value == "" or value == []:
            return False

        # Case-insensitive check for specific strings
        if isinstance(value, str) and value.lower() in ["nan", "not specified"]:
            return False

        # If value is a list, check each item
        if isinstance(value, list):
            for item in value:
                # Check if the item is a string and matches the specified values
                if isinstance(item, str) and item.lower() in ["nan", "not specified"]:
                    return False

        return True

    # Optimize 'city' and 'country'
    # If location_city is valid use it ('country'='JP" and location_country='Japan')
    for key in ["city", "country"]:
        location_model_key = f"location_model_{key}"
        # if not is_valid(data_dict.get(key)) and is_valid(
        #     data_dict.get(location_model_key)
        # ):
        if is_valid(data_dict.get(location_model_key)):
            data_dict[key] = data_dict[location_model_key]
        data_dict.pop(location_model_key, None)

    # Optimize 'ai' and 'involves_ai'
    if data_dict.get("ai") not in [0, 1] and is_valid(data_dict.get("involves_ai")):
        data_dict["ai"] = 1 if data_dict["involves_ai"] else 0
    data_dict.pop("involves_ai", None)
    if data_dict.get("ai") in [0, 1]:
        data_dict["ai"] = bool(data_dict["ai"])

    # Optimize 'salary_numerical'
    salary = data_dict.get("salary_numerical")
    print("salary", salary)
    if not is_valid(salary) or isinstance(salary, str):
        data_dict["salary_min"] = data_dict["salary_max"] = None
    elif (
        isinstance(salary, list)
        and len(salary) == 1
        and is_valid(salary[0])
        and salary[0] < 900000
        and salary[0] > 5
    ):
        data_dict["salary_min"] = data_dict["salary_max"] = salary[0]
    elif (
        isinstance(salary, list)
        and len(salary) == 2
        and all(is_valid(val) for val in salary)
        and all(val < 900000 for val in salary)
        and all(val > 5 for val in salary)
    ):
        data_dict["salary_min"], data_dict["salary_max"] = salary
    else:
        data_dict["salary_min"] = data_dict["salary_max"] = None

    print("salary_min", data_dict["salary_min"])
    print("salary_max", data_dict["salary_max"])
    print("salary_frequency", data_dict["salary_frequency"], "\n")
    data_dict.pop("salary_numerical", None)

    # Optimize 'job_skills'
    if is_valid(data_dict.get("required_skills")) and not is_valid(data_dict.get("skills")):
        data_dict["skills"] = data_dict["required_skills"]
    elif is_valid(data_dict.get("skills")):
        data_dict["skills"] = data_dict.get("skills").split(",")
    data_dict.pop("required_skills", None)

    # Optimize 'experience_years'
    if is_valid(data_dict.get("experience_years")):
        experience_years = data_dict["experience_years"]
        if isinstance(experience_years, str):
            data_dict["experience_years"] = None
        else:
            data_dict["experience_years"] = int(experience_years)

    # Replace 'Not specified' with None for all keys
    for key, value in data_dict.items():
        if not is_valid(value):
            data_dict[key] = None

    # Transform lists into strings
    for key, value in data_dict.items():
        if isinstance(value, list):
            data_dict[key] = ", ".join(value) if len(value) > 1 else value[0]

    # Transform values into lowercase strings
    lowercased_data = {
        key: value.lower() if isinstance(value, str) else value
        for key, value in data_dict.items()
    }

    return lowercased_data

In [ ]:
list_of_dicts = []
for i, dict in enumerate(flattened_data):
    print("index", i)
    optimized_dict = optimize_keys(dict)
    list_of_dicts.append(optimized_dict)

# print(list_of_dicts[8978].keys())
print(list_of_dicts[8978])
# for i, dict in enumerate(list_of_dicts):
# print(list_of_dicts[2])
# print(dict.keys())
# if i == 5:
# break

In [ ]:
import pandas as pd
import tiktoken
import logging
logger = logging.getLogger(__name__)

# The data in a dataframe
df1 = pd.DataFrame(list_of_dicts)
df1 = df1.rename(columns={"ai": "involves_ai"})
df1 = df1.rename(columns={"title": "job_title"})
df1 = df1.rename(columns={"skills": "job_skills"})

df1['created_at'] = pd.to_datetime(df1['created_at'])
df1 = df1.rename(columns={"created_at": "creation_date"})

# df1 = df1.rename(columns={"cleaned_description": "job_listing_text"})
df1 = df1.drop(
    ["slug", "approved", "chain_of_thought", "job_type_reasoning", "remote_reasoning"],
    axis=1,
)

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    try:
        encoding = tiktoken.get_encoding(encoding_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens
    except Exception as e:
        logger.error(f"Error calculating number of tokens: {e}")
        return 0

df1["num_tokens"] = df1["job_listing_text"].apply(
        lambda x: num_tokens_from_string(x, "cl100k_base")
    )

# # Finding and printing the max value in the salary_max column
# max_salary_max = df1['salary_max'].max()
# print(f"Maximum value in 'salary_max' column: {max_salary_max}")

# # Finding and printing the min value in the salary_min column
# min_salary_min = df1['salary_min'].min()
# print(f"Minimum value in 'salary_min' column: {min_salary_min}")

# # Print the row with the max salary_max value
# print("Row with the maximum 'salary_max' value:")
# print(df1[df1['salary_max'] == max_salary_max])

# # Print the row with the min salary_min value
# print("\nRow with the minimum 'salary_min' value:")
# print(df1[df1['salary_min'] == min_salary_min])

# print(df1.head(1))
# print(df1.dtypes)
# print("\n")
print(df1.info())
# print(df1.shape)
# print(df1.columns)
# print(len(df1))

# df1.to_pickle("../data/extracted_cleaned_df_feb5.pkl")
# print(df1.to_markdown())

In [2]:
import pandas as pd
df = pd.read_json("db_info.json")
print(df.info())
# print(df.head(1))

# Get all unique values in the 'job_type' column
unique_job_types = df['job_type'].unique()

# Print the unique job types
print(unique_job_types)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7907 entries, 0 to 7906
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   job_id                   7907 non-null   int64         
 1   created_at               7907 non-null   datetime64[ns]
 2   job_title                7907 non-null   object        
 3   job_skills               7907 non-null   object        
 4   job_type                 7907 non-null   object        
 5   company_id               7907 non-null   int64         
 6   apply_url                7907 non-null   object        
 7   city                     7907 non-null   object        
 8   country                  7907 non-null   object        
 9   salary                   7907 non-null   object        
 10  salary_min               7907 non-null   int64         
 11  salary_max               7907 non-null   int64         
 12  salary_currency          7907 non-